<a href="https://colab.research.google.com/github/mcstllns/DeepLearning_2025/blob/main/PRACTICA_04_Clasificacion_con_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font color="darkorange" size="10"><b>04. Clasificación con MLP</b></font>

Miguel A. Castellanos

En esta práctica vamos a hacer clasificaciones binarias y múltiples

Son similares a las hechas con playground pero ahora crearemos y controlaremos nosotros los datos.

In [ ]:
# Vamos a empezar con 2 pelotas

from sklearn.datasets import make_blobs
from matplotlib import pyplot as plt

X, y = make_blobs(centers=2, cluster_std=0.8)

plt.scatter(X[y == 0, 0], X[y == 0, 1], label="0")
plt.scatter(X[y == 1, 0], X[y == 1, 1], label="1")
plt.legend()

In [ ]:
# Vamos a instalar torcheval, es parte de torch y va a facilitar la evaluacion del modelo

!pip install torcheval

In [ ]:
# Preparamos los datos como antes, pero y no hay que normalizarla
import torch
from torch.utils.data import TensorDataset, DataLoader

# Normalizamos solo X

x_mean, x_std = X.mean(), X.std()
X_norm = (X - x_mean) / x_std

tensor_X = torch.Tensor(X_norm)
tensor_y = torch.Tensor(y)

my_dataset = TensorDataset(tensor_X,tensor_y) # create your datset
my_dataloader = DataLoader(my_dataset) # create your dataloader

In [ ]:
# definimos mi red igual que antes
# Es una red con 3 capas (50,25,1), todo relu menos la ultima capa que hace
# la clasificación binaria con una sigmoide

class MLP(torch.nn.Module):
    def __init__(self, num_features):
        super().__init__()

        self.all_layers = torch.nn.Sequential(

            # 1st hidden layer
            torch.nn.Linear(num_features, 50),
            torch.nn.ReLU(),

            # 2nd hidden layer
            torch.nn.Linear(50, 25),
            torch.nn.ReLU(),

            # output layer
            torch.nn.Linear(25, 1),
            # torch.nn.Sigmoid() # ver comentario 1

        )

    def forward(self, x):
        output = self.all_layers(x)
        # return output
        return output.flatten() # ver comentario 2

## Comentario 1

tenemos dos opciones, o definimos en la última capa la función sigmoid y usamos como función de costo BCELoss

```python
torch.nn.Sigmoid()
loss_fn = torch.nn.BCELoss()
```

O no definimmos sigmoid y usamos BCEWithLogitsLoss

```python
# torch.nn.Sigmoid()
loss_fn = torch.nn.BCEWithLogitsLoss()
```


Esta segunda opción funciona mucho mejor pero hay que tener en cuenta que va a devolver valores que no están entre [0,1] sino [-ing,+inf]


## Comentario 2
Ojo con las dimnsiones de entrada, salida y las internas
Por ejemplo en este codigo la red devuelve una lista (10,1) (una matriz) y los rasgos son (10,) (un vector)

```python
def forward(self, x):
  output = self.all_layers(x)
  return output
```

Siempre hay funciones que necesitan una cosa pero tú les pasas otra y hay que ajustarlas, en este caso hemos optado por "aplanar" la salida de la red para que coincida con el cálculo del accurary

```python
def forward(self, x):
  output = self.all_layers(x)
  return output.flatten()
```


In [ ]:
from torcheval.metrics import BinaryAccuracy, BinaryConfusionMatrix

# definimos hiper-parámetros

torch.manual_seed(1)

model = MLP(num_features=2) # instanciamos el modelo

# loss_fn = torch.nn.BCELoss() # Ojo, ahora es BCE, necesita sigmoid
loss_fn = torch.nn.BCEWithLogitsLoss() # Ojo, ahora es BCELog, no necesita sigmoid

optimizer = torch.optim.Adam(model.parameters(), lr=0.001) # usamos Adam

acc = BinaryAccuracy() # calculamos la precisión (accurary)
m_confusion = BinaryConfusionMatrix(threshold=0.5)


In [ ]:
num_epochs = 10

# vamos a guardar el lost pero tambien el accuracy
loss_list, acc_list = [], []

for epoch in range(num_epochs):

    model.train()
    for batch_idx, (features, targets) in enumerate(my_dataloader):

        # forward
        output = model(features)
        loss = loss_fn(output, targets)

        # backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Esto es algo especifico para la evaluacion, en cada epoca actualizamos:
        acc.update(output, targets)      # el accuracy

        # añadimos los nuevos valores a la lista
        loss_list.append(loss.item())
        acc_list.append(acc.compute().item())

        if not batch_idx % 100:
            ### Imprimimos info
            print(
                f"Epoch: {epoch+1:03d}/{num_epochs:03d}"
                f" | Batch {batch_idx:03d}/{len(my_dataloader):03d}"
                f" | Train Loss: {loss:.2f}"
                f" | Accuracy: {acc.compute():.2f}"
            )


In [ ]:
# Vemos que predicciones hace nuestro modelo para nuestro X

model.eval()
yp = model(tensor_X)
yp

In [ ]:
# Calculamos una matriz de confusion

k = BinaryConfusionMatrix(threshold=0.)
k.update(yp, tensor_y.long())
print(k.compute())

In [ ]:
# Ploteamos

plt.plot(loss_list)
plt.plot(acc_list)
plt.legend(['loss', 'accuracy'])
plt.show()

---
# Ejercicios

Crea un notebook de colab con las soluciones a los ejercicios, hazlo público y sube el enlace al campus virtual


### 01. Ejercicio 1

Usando el generador de pelotas make_blobs crea unos datos con 4 categorías y calcula un MLP para hacer la clasificación.

Ahora tendrás que modificar como ploteas

Y también la última función de activación de la red será una SoftMax

Al igual que con BCE, se prefiere la opción de hacer implicita esas funciones de activación y que sea la función que calcula la función de coste la que las incluya, es decir:



```python

# no se pone la softmax porque va incluida en el optimizador
# torch.nn.Softmax(dim=4)

...

# ya incluye automaticamente las softmax al definir fn_loss
loss_fn = torch.nn.CrossEntropyLoss()  
```



In [ ]:
X, y = make_blobs(centers=4, cluster_std=0.8)

plt.scatter(X[y == 0, 0], X[y == 0, 1], label="0")
plt.scatter(X[y == 1, 0], X[y == 1, 1], label="1")
plt.scatter(X[y == 2, 0], X[y == 2, 1], label="2")
plt.scatter(X[y == 3, 0], X[y == 3, 1], label="3")
plt.legend()
plt.show()

pasa lo mismo, no se pone la capa de softmax, va implicita al definir la funcion criterion = nn.CrossEntropyLoss()  # Combina softmax y NLLLoss



ahora la matriz de confusion es MulticlassConfusionMatrix(4)

### 02. Ejercicio 2

Haz lo mismo usando make_circles

```python
from sklearn.datasets import make_circles
```


### 03. Ejercicio 3

Haz lo mismo usando make_moons

```python
from sklearn.datasets import make_moons
```

### 04. Ejercicio 04. Clasificación binaria con datos de infidelidad

Se analizan unos datos obtenido de Kaggle.com sobre una encuesta de infidelidad: [enlace](https://www.google.com/url?q=https%3A%2F%2Fwww.kaggle.com%2Fdatasets%2Futkarshx27%2Ffairs-extramarital-affairs-data)

Las variables son:

**Criterio**: affairs.

**Predictoras**: gender, age, yearsmarried, children, religiousness, education, occupation, rating.

La variable criterio es de tipo dicotómico **0 = no, 1 = sí** por lo que el análisis convencional nos lleva a un modelo binomial. La precisión (accuracy) en la clasificación obtenida con este modelo es de 0.72

Construye un MLP que supere la precisión del modelo lineal

In [ ]:
import pandas as pd

url = 'https://raw.githubusercontent.com/mcstllns/UNIR2024/main/data-affairs.csv'
data  = pd.read_csv(url)
print(data.keys())
data.head()


In [ ]:
X = data.drop('affairs', axis=1)
y = data['affairs']

print(X.shape)
print(y.shape)